In [1]:
import pandas as pd
import numpy as np
from google.colab import drive
from geopy.geocoders import Nominatim
from geopy.point import Point
import warnings
warnings.filterwarnings('ignore')

In [2]:
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
def make_df_dtype(data):
    '''
    Создает датафрейм для анализа исходного фрейма по каждому признаку.
    Каждая строка соответствует столбцу исходного датафрейма и предоставляет информацию:
    - тип данных
    - количество уникальных значений
    - процент пропущенных значений

    Args:
        data (DataFrame): Датафремй для анализа

    Returns:
        df(DataFrame): Сформированный датафрейм
    '''

    cols, dtype_col, nunique, null_share = [], [], [], []
    for column in data:
        cols.append(column)

        this_dtype = data[column].dtype
        dtype_col.append(this_dtype)

        nunique.append(data[column].nunique())

        null_sum = data[column].isna().sum()
        null_to_len = null_sum / len(data[column])
        null_share.append(f'{null_to_len:.2%}')

    df = pd.DataFrame(list(zip(dtype_col,
                               nunique,
                              null_share)),
                      index=cols)
    df.columns=['dtype',
                'nunique',
               'null_share']

    return df


In [4]:
def get_country(row):
  '''
    Определяет страну по координатам через обращение к сервису geolocator.

    Args:
        row (pandas.core.series.Series): строка датафрейма

    Returns:
        наименование страны или np.nan в случае ошибки
    '''
    #pos = str(row['lat']) + ', ' + str(row['lng'])
  geolocator = Nominatim(user_agent="http")
  try:
      location = geolocator.reverse(Point(row['lat'], row['lng']), timeout = 10)
      return location.raw.get('address').get('country')
  except ValueError:
      return np.nan

In [5]:
def validate_country(row):
  '''
    Определяет страну по координатам через обращение к сервису geolocator.

    Args:
        row (pandas.core.series.Series): строка датафрейма

    Returns:
        наименование страны или np.nan в случае ошибки
    '''
    #pos = str(row['lat']) + ', ' + str(row['lng'])
  geolocator = Nominatim(user_agent="http")
  #dfLat.loc[dfLat.duplicated(subset=dfLat.columns.difference(['lat', 'lng']), keep=False)]
  try:
      location = geolocator.reverse(Point(row['lat'], row['lng']), timeout = 10)
      res = location.raw.get('address').get('country')
      if (res==row['Country']):
        return res
      else:
        return np.nan
  except ValueError:
      return np.nan

# Предобработка дополнительного датафрейма

# Работа с пропусками

In [6]:
dfLat = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Проект/AQI and Lat Long of Countries.csv')

Этот датарейм используется для заполнения пропущенных стран и широты и долготы в рабочем датафрейме. Прежде чем объединять датасеты слева, я выкину из датасета №2 все строки с пустой долготой и широтой, так как они нам не нужны.

In [7]:
dfLat.dropna(subset = ['lat', 'lng'], inplace=True)

Скорректируем названия стран.

In [19]:
!pip install country_converter --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 853.8 kB/s eta 0:00:00


The input format is determined automatically, based on ISO two letter, ISO three letter, ISO numeric or regular expression matching. In case of any ambiguity, the source format can be specified with the parameter 'src'.

In [29]:
dfLat.Country.unique()

array(['Russian Federation', 'Brazil', 'Italy', 'Poland',
       'United States of America', 'Belgium', 'China', 'Netherlands',
       'France', 'Colombia', 'Romania', 'Indonesia', 'Finland',
       'South Africa',
       'United Kingdom of Great Britain and Northern Ireland', 'India',
       'Haiti', 'Philippines', 'Latvia', 'Germany', 'New Zealand',
       'Mexico', 'Japan', 'El Salvador', 'Nigeria', 'South Sudan',
       'Ireland', 'Turkey', 'Democratic Republic of the Congo', 'Canada',
       'Switzerland', 'Denmark', 'Cameroon', 'Australia', 'Portugal',
       'Ethiopia', 'Hungary', 'Iran (Islamic Republic of)', 'Israel',
       "Côte d'Ivoire", 'Papua New Guinea', 'Madagascar', 'Ghana',
       'Armenia', 'Guatemala', 'Pakistan', 'Thailand', 'Malaysia',
       'Ecuador', 'Argentina', 'Peru', 'Dominican Republic', 'Azerbaijan',
       'Spain', 'Guinea', 'Bolivia (Plurinational State of)', 'Chile',
       'Republic of North Macedonia', 'Kazakhstan', 'Serbia', 'Egypt',
       'Czechi

In [32]:
import country_converter as coco
cc = coco.CountryConverter()
dfLat['Country'] = dfLat['Country'].apply(lambda x: x if pd.isna(x) else cc.convert(names=x, to='name_short'))

Стоит обратить внимание на страну которая изначально была записана как Viet Nam)

In [34]:
dfLat.Country.unique()

array(['Russia', 'Brazil', 'Italy', 'Poland', 'United States', 'Belgium',
       'China', 'Netherlands', 'France', 'Colombia', 'Romania',
       'Indonesia', 'Finland', 'South Africa', 'United Kingdom', 'India',
       'Haiti', 'Philippines', 'Latvia', 'Germany', 'New Zealand',
       'Mexico', 'Japan', 'El Salvador', 'Nigeria', 'South Sudan',
       'Ireland', 'Türkiye', 'DR Congo', 'Canada', 'Switzerland',
       'Denmark', 'Cameroon', 'Australia', 'Portugal', 'Ethiopia',
       'Hungary', 'Iran', 'Israel', "Cote d'Ivoire", 'Papua New Guinea',
       'Madagascar', 'Ghana', 'Armenia', 'Guatemala', 'Pakistan',
       'Thailand', 'Malaysia', 'Ecuador', 'Argentina', 'Peru',
       'Dominican Republic', 'Azerbaijan', 'Spain', 'Guinea', 'Bolivia',
       'Chile', 'North Macedonia', 'Kazakhstan', 'Serbia', 'Egypt',
       'Czechia', 'Malawi', 'Sierra Leone', 'Ukraine', 'Benin', 'Namibia',
       'Albania', nan, 'Guyana', 'Senegal', 'Lesotho', 'Venezuela',
       'Bulgaria', 'Somalia', 'Solo

Проверим есть ли дубликаты без учета широт.

In [35]:
dup_rows = dfLat.loc[dfLat.duplicated(subset=dfLat.columns.difference(['lat', 'lng']), keep=False)]
dup_rows.shape[0]

3809

Добавим дополнительный признак в доп датасет: Дубликат. Для этого воспользуемся функцией. Если наша строка есть в списке дубликатов, то мы ее отмечаем. В дальнейшем это поможет нам убрать некорректные значения стран.

In [36]:
dfLat.loc[dfLat.duplicated(subset=dfLat.columns.difference(['lat','lng']), keep=False), 'dup'] = 'dup'
dfLat.loc[~dfLat.duplicated(subset=dfLat.columns.difference(['lat','lng']), keep=False), 'dup'] = 'Nor'

Сделаем валидацию стран вспомогательного датасета. Почему? Потому что нам все равно могут потребоваться координаты. так как мы уже отметили те строки которые являются дублями, но не по координатам, а таких почти 1/4 датасета, то нам надо провалидировать лишь их. Мы возьмем все строки, в которых есть такая отметка и отправим их на проверку страны.
Берем координаты, отправляем их в валидирующую функцию, которая возвращает либо наименование страны, если координаты совпали, либо возвращает Nan

In [37]:
dfLat['Country'] = dfLat.apply(lambda row: validate_country(row) if (row['dup']=='dup') else row['Country'], axis = 1)

In [38]:
dup_rows = dfLat.loc[dfLat.duplicated(subset=dfLat.columns.difference(['lat', 'lng']), keep=False)]
dup_rows.shape[0]

3225

In [39]:
dup_rows.head(6)

,Country,City,AQI Value,AQI Category,CO AQI Value,CO AQI Category,Ozone AQI Value,Ozone AQI Category,NO2 AQI Value,NO2 AQI Category,PM2.5 AQI Value,PM2.5 AQI Category,lat,lng,dup
1,NaN,Presidente Dutra,41,Good,1,Good,5,Good,1,Good,41,Good,-5.2900,-44.4900,dup
2,NaN,Presidente Dutra,41,Good,1,Good,5,Good,1,Good,41,Good,-11.2958,-41.9869,dup
20,United States,Pontiac,77,Moderate,2,Good,22,Good,15,Good,77,Moderate,42.6493,-83.2878,dup
21,United States,Pontiac,77,Moderate,2,Good,22,Good,15,Good,77,Moderate,40.8894,-88.6409,dup
33,United States,Dayton,45,Good,1,Good,32,Good,7,Good,45,Good,39.7805,-84.2003,dup
34,United States,Dayton,45,Good,1,Good,32,Good,7,Good,45,Good,39.2592,-119.5653,dup


Остались строки в которых координаты максимально близко, удалить можно любые дубликаты, оставив первый отмеченный как False. Избавимся сначала от проверенных строк с пустой страной и удалим их. (То есть он где-то 1000 строк проверил и вернул 500 пустых строк, и 500 верных). В остальных страны определены верно.

In [40]:
dfLat = dfLat.drop_duplicates(subset=dfLat.columns.difference(['lat', 'lng']), keep='first')

Отлично, мы почистили вспомогательный датасет. Проверим какие колонки в нем есть, чтобы сразу удалить ненужные.

In [41]:
dfLat.dtypes

Country                object
City                   object
AQI Value               int64
AQI Category           object
CO AQI Value            int64
CO AQI Category        object
Ozone AQI Value         int64
Ozone AQI Category     object
NO2 AQI Value           int64
NO2 AQI Category       object
PM2.5 AQI Value         int64
PM2.5 AQI Category     object
lat                   float64
lng                   float64
dup                    object
dtype: object

In [42]:
dfLat.drop(['dup'], axis=1)

,Country,City,AQI Value,AQI Category,CO AQI Value,CO AQI Category,Ozone AQI Value,Ozone AQI Category,NO2 AQI Value,NO2 AQI Category,PM2.5 AQI Value,PM2.5 AQI Category,lat,lng
0,Russia,Praskoveya,51,Moderate,1,Good,36,Good,0,Good,51,Moderate,44.7444,44.2031
1,NaN,Presidente Dutra,41,Good,1,Good,5,Good,1,Good,41,Good,-5.2900,-44.4900
3,Italy,Priolo Gargallo,66,Moderate,1,Good,39,Good,2,Good,66,Moderate,37.1667,15.1833
4,Poland,Przasnysz,34,Good,1,Good,34,Good,0,Good,20,Good,53.0167,20.8833
5,NaN,Punta Gorda,54,Moderate,1,Good,14,Good,11,Good,54,Moderate,16.1005,-88.8074
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16689,China,Shaoguan,160,Unhealthy,3,Good,160,Unhealthy,1,Good,79,Moderate,24.8011,113.5927
16690,United States,Highland Springs,54,Moderate,1,Good,34,Good,5,Good,54,Moderate,37.5516,-77.3285
16691,NaN,Martin,71,Moderate,1,Good,39,Good,1,Good,71,Moderate,49.0650,18.9219
16693,France,Sceaux,50,Good,1,Good,20,Good,5,Good,50,Good,48.7786,2.2906


Подгрузим основной даасет и сделаем аналогичную проверку названий стран.

In [43]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Проект/global air pollution dataset.csv')

In [44]:
data.Country.nunique()

175

In [45]:
cc = coco.CountryConverter()
data['Country'] = data['Country'].apply(lambda x: x if pd.isna(x) else cc.convert(names=x, to='name_short'))

In [46]:
data.Country.nunique()

175

Выведем топ 10 с пропущенными странами, чтобы посмотреть по какой причине они пропущенны.

In [47]:
data.loc[(data.Country.isna())].head(10)

,Country,City,AQI Value,AQI Category,CO AQI Value,CO AQI Category,Ozone AQI Value,Ozone AQI Category,NO2 AQI Value,NO2 AQI Category,PM2.5 AQI Value,PM2.5 AQI Category
535,NaN,Granville,30,Good,1,Good,30,Good,3,Good,25,Good
654,NaN,Kingston Upon Hull,33,Good,1,Good,17,Good,8,Good,33,Good
787,NaN,New Waterford,20,Good,1,Good,18,Good,9,Good,20,Good
801,NaN,Kingstown,163,Unhealthy,0,Good,25,Good,0,Good,163,Unhealthy
906,NaN,Nanakuli,30,Good,0,Good,27,Good,0,Good,30,Good
916,NaN,Lavagna,55,Moderate,1,Good,38,Good,2,Good,55,Moderate
968,NaN,Ladispoli,48,Good,1,Good,48,Good,2,Good,34,Good
985,NaN,Dong Hoi,55,Moderate,0,Good,26,Good,0,Good,55,Moderate
1139,NaN,Nettuno,53,Moderate,1,Good,53,Moderate,1,Good,23,Good
1169,NaN,Puebloviejo,71,Moderate,1,Good,16,Good,0,Good,71,Moderate


Судя по результатам, многие значения просто написаны с не с той буквы. Например Nanakuli, там используются дифтонги. Или в данных опечатка. Мне не хочется удалять такое количество строк из сета, все таки это почти два процента. Поэтому я воспользуюсь вторым датасетом, который был упомянут в Readme. Собственно он для этой цели и планировался. Я объединю два датасета, чтобы получить координаты. А далее по координатам можно будет определить страну.

In [48]:
data = data.merge(dfLat, how='left')

In [49]:
data.shape

(23463, 15)

Подтянула лишнюю колонку с дублями

In [50]:
data.isna().sum()

Country                427
City                     1
AQI Value                0
AQI Category             0
CO AQI Value             0
CO AQI Category          0
Ozone AQI Value          0
Ozone AQI Category       0
NO2 AQI Value            0
NO2 AQI Category         0
PM2.5 AQI Value          0
PM2.5 AQI Category       0
lat                   9693
lng                   9693
dup                   9693
dtype: int64

Как видно, количество неизвестных стран осталось тем же, зато теперь у нас есть координаты и мы можем восопльзоваться геосервисом

In [51]:
pip install geopy

In [52]:

data['Country'] = data.apply(lambda row: get_country(row) if pd.isnull(row['Country']) else row['Country'], axis = 1)

Проверим заполнились ли данные на примере Nanakuli

In [53]:
data.loc[(data.City=='Nanakuli')]

,Country,City,AQI Value,AQI Category,CO AQI Value,CO AQI Category,Ozone AQI Value,Ozone AQI Category,NO2 AQI Value,NO2 AQI Category,PM2.5 AQI Value,PM2.5 AQI Category,lat,lng,dup
906,United States,Nanakuli,30,Good,0,Good,27,Good,0,Good,30,Good,21.3892,-158.1445,Nor


In [54]:
dup_rows = data.loc[data.duplicated(subset=data.columns.difference(['lat', 'lng']), keep=False)]
dup_rows.shape[0]

0

In [55]:
(data.loc[(data.Country.isna())].shape[0]/data.shape[0])*100

0.6563525550867323

In [56]:
data.isna().sum()

Country                154
City                     1
AQI Value                0
AQI Category             0
CO AQI Value             0
CO AQI Category          0
Ozone AQI Value          0
Ozone AQI Category       0
NO2 AQI Value            0
NO2 AQI Category         0
PM2.5 AQI Value          0
PM2.5 AQI Category       0
lat                   9693
lng                   9693
dup                   9693
dtype: int64

так, у нас осталось 154 строки без страны и координат. Города там написаны явно с грамматической ошибкой. И процент таких строк 0,65%. То мы можем смело их удалить.А вот пустые строки с координатами нас не так сильно беспокоят. Их можно будет заполнить с помощью того же сервиса geopy, если они потребуются

In [57]:
data.dropna(subset = ['Country'], inplace=True)

Поле объединения датасетов у нас немного дублей появилось, надо их удалить

In [58]:
data['City'].nunique()

23308

In [59]:
data.drop_duplicates(subset=['City'])

,Country,City,AQI Value,AQI Category,CO AQI Value,CO AQI Category,Ozone AQI Value,Ozone AQI Category,NO2 AQI Value,NO2 AQI Category,PM2.5 AQI Value,PM2.5 AQI Category,lat,lng,dup
0,Russia,Praskoveya,51,Moderate,1,Good,36,Good,0,Good,51,Moderate,44.7444,44.2031,Nor
1,Brazil,Presidente Dutra,41,Good,1,Good,5,Good,1,Good,41,Good,NaN,NaN,NaN
2,Italy,Priolo Gargallo,66,Moderate,1,Good,39,Good,2,Good,66,Moderate,37.1667,15.1833,Nor
3,Poland,Przasnysz,34,Good,1,Good,34,Good,0,Good,20,Good,53.0167,20.8833,Nor
4,France,Punaauia,22,Good,0,Good,22,Good,0,Good,6,Good,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23458,India,Gursahaiganj,184,Unhealthy,3,Good,154,Unhealthy,2,Good,184,Unhealthy,NaN,NaN,NaN
23459,France,Sceaux,50,Good,1,Good,20,Good,5,Good,50,Good,48.7786,2.2906,Nor
23460,India,Mormugao,50,Good,1,Good,22,Good,1,Good,50,Good,NaN,NaN,NaN
23461,United States,Westerville,71,Moderate,1,Good,44,Good,2,Good,71,Moderate,40.1241,-82.9210,Nor




---



---



---
Дополним данные координатами для полного отображения данных на карте


In [60]:
data.tail()

,Country,City,AQI Value,AQI Category,CO AQI Value,CO AQI Category,Ozone AQI Value,Ozone AQI Category,NO2 AQI Value,NO2 AQI Category,PM2.5 AQI Value,PM2.5 AQI Category,lat,lng,dup
23458,India,Gursahaiganj,184,Unhealthy,3,Good,154,Unhealthy,2,Good,184,Unhealthy,NaN,NaN,NaN
23459,France,Sceaux,50,Good,1,Good,20,Good,5,Good,50,Good,48.7786,2.2906,Nor
23460,India,Mormugao,50,Good,1,Good,22,Good,1,Good,50,Good,NaN,NaN,NaN
23461,United States,Westerville,71,Moderate,1,Good,44,Good,2,Good,71,Moderate,40.1241,-82.9210,Nor
23462,Malaysia,Marang,70,Moderate,1,Good,38,Good,0,Good,70,Moderate,NaN,NaN,NaN


In [61]:
data.isna().sum()

Country                  0
City                     1
AQI Value                0
AQI Category             0
CO AQI Value             0
CO AQI Category          0
Ozone AQI Value          0
Ozone AQI Category       0
NO2 AQI Value            0
NO2 AQI Category         0
PM2.5 AQI Value          0
PM2.5 AQI Category       0
lat                   9539
lng                   9539
dup                   9539
dtype: int64

In [62]:
from geopandas.tools import geocode
for index,row in data.iterrows():
  if pd.isna(row['lng']):
    try:
      information = geocode(row['City'], provider = 'nominatim', user_agent='xyz', timeout=5)
      data.loc[index, 'lng'] = information.geometry.loc[0].x
      data.loc[index,'lat'] = information.geometry.loc[0].y
    except:
      print('Coordinates of '+row['City']+ ' are not available')

Coordinates of West Carrollton City are not available
Coordinates of Goryachevodskiy are not available
Coordinates of Sungaipenuh are not available
Coordinates of Taburbah are not available
Coordinates of Ballitoville are not available
Coordinates of Chilakalurupet are not available
Coordinates of Kryvyy Rih are not available
Coordinates of Izyaslav are not available
Coordinates of Nangloi Jat are not available
Coordinates of Naravarikuppam are not available
Coordinates of Lopatinskiy are not available
Coordinates of Muvattupula are not available
Coordinates of Masjed E Soleyman are not available
Coordinates of Houlung are not available
Coordinates of Phu Khieo are not available
Coordinates of Dembi Dolo are not available
Coordinates of Sattankulam are not available
Coordinates of Yandoon are not available
Coordinates of Dokshukino are not available
Coordinates of Energeticheskiy are not available
Coordinates of Boshkengash are not available
Coordinates of Sannicolaul Mare are not avai

Coordinates of Tecoman are not available
Coordinates of Thivai are not available
Coordinates of Tripoli are not available
Coordinates of Tuysarkan are not available
Coordinates of Viransehir are not available
Coordinates of Pandaria are not available
Coordinates of Parichhatgarh are not available
Coordinates of Rabwah are not available
Coordinates of Taklisah are not available
Coordinates of Tastur are not available
Coordinates of Kaliwungu are not available
Coordinates of Escazu are not available
Coordinates of Boldumsaz are not available
Coordinates of Boshan are not available
Coordinates of Bousse are not available
Coordinates of Chambishi are not available
Coordinates of Bene Beraq are not available
Coordinates of Maymanak are not available
Coordinates of Mehndawal are not available
Coordinates of Kahna Nau are not available
Coordinates of Homnabad are not available
Coordinates of Hsinpu are not available
Coordinates of Karsiyang are not available
Coordinates of Nkowakowa are not a

Coordinates of Sangareddi are not available
Coordinates of Kot Putli are not available
Coordinates of Piriyapatna are not available
Coordinates of Bardaw are not available
Coordinates of Khalandrion are not available
Coordinates of Hsinhua are not available
Coordinates of Moji Das Cruzes are not available
Coordinates of Itimadpur are not available
Coordinates of Novosilikatnyy are not available
Coordinates of Honavar are not available
Coordinates of Garui are not available
Coordinates of Fiditi are not available
Coordinates of Kasimbazar are not available
Coordinates of Kaikalur are not available
Coordinates of Kuanhsi are not available
Coordinates of Nyzhnya Krynka are not available
Coordinates of Virarajendrapet are not available
Coordinates of Oraiokastron are not available
Coordinates of Pangkalanbuun are not available
Coordinates of Pinjaur are not available
Coordinates of Borujan are not available
Coordinates of Yungkang are not available
Coordinates of Chuhar Kana are not availa

In [84]:
data.isna().sum()

Country                  0
City                     0
AQI Value                0
AQI Category             0
CO AQI Value             0
CO AQI Category          0
Ozone AQI Value          0
Ozone AQI Category       0
NO2 AQI Value            0
NO2 AQI Category         0
PM2.5 AQI Value          0
PM2.5 AQI Category       0
lat                    286
lng                    286
dup                   9539
dtype: int64

Посмотрим, на пустые строки. Сначала на город:

In [78]:
data = data.reset_index(drop=True)

In [79]:
data.loc[pd.isna(data['City'])]

,Country,City,AQI Value,AQI Category,CO AQI Value,CO AQI Category,Ozone AQI Value,Ozone AQI Category,NO2 AQI Value,NO2 AQI Category,PM2.5 AQI Value,PM2.5 AQI Category,lat,lng,dup
2628,Israel,NaN,68,Moderate,1,Good,48,Good,4,Good,68,Moderate,46.314475,11.048029,NaN


Его можно поправить вруную.

In [81]:
data['City'][2628]= 'Jerusalem'

In [82]:
data.iloc[[2628]]

,Country,City,AQI Value,AQI Category,CO AQI Value,CO AQI Category,Ozone AQI Value,Ozone AQI Category,NO2 AQI Value,NO2 AQI Category,PM2.5 AQI Value,PM2.5 AQI Category,lat,lng,dup
2628,Israel,Jerusalem,68,Moderate,1,Good,48,Good,4,Good,68,Moderate,46.314475,11.048029,NaN


In [85]:
data.drop(['dup'], axis=1, inplace=True)

Теперь посмотрим что можно сделать с пустыми координатами для городов в названии которых есть ошибка.

In [87]:
data.loc[pd.isna(data['lat'])]

,Country,City,AQI Value,AQI Category,CO AQI Value,CO AQI Category,Ozone AQI Value,Ozone AQI Category,NO2 AQI Value,NO2 AQI Category,PM2.5 AQI Value,PM2.5 AQI Category,lat,lng
153,United States,West Carrollton City,84,Moderate,3,Good,9,Good,24,Good,84,Moderate,NaN,NaN
179,Russia,Goryachevodskiy,48,Good,1,Good,39,Good,1,Good,48,Good,NaN,NaN
193,Indonesia,Sungaipenuh,17,Good,1,Good,13,Good,0,Good,17,Good,NaN,NaN
199,Tunisia,Taburbah,49,Good,1,Good,49,Good,1,Good,32,Good,NaN,NaN
278,South Africa,Ballitoville,222,Very Unhealthy,12,Good,5,Good,14,Good,222,Very Unhealthy,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22999,Albania,Kamze,60,Moderate,1,Good,44,Good,2,Good,60,Moderate,NaN,NaN
23073,Greece,Aiyion,78,Moderate,1,Good,40,Good,0,Good,78,Moderate,NaN,NaN
23160,India,Palkonda,110,Unhealthy for Sensitive Groups,2,Good,35,Good,3,Good,110,Unhealthy for Sensitive Groups,NaN,NaN
23187,India,Dhupgari,154,Unhealthy,5,Good,15,Good,11,Good,154,Unhealthy,NaN,NaN


Принимая во внимание объем выборки, объем подвыборки с нехаполненными координатами, а также какие именно города попали в подвыборку - принято решение удалить данную подвыборку.

In [88]:
data.shape

(23309, 14)

In [89]:
data= data.dropna()

In [90]:
data.shape

(23023, 14)

In [91]:
data.to_csv('finalData.csv')